# Packages 

In [1]:
%load_ext autoreload
%autoreload 2
import sys
import logging
base_dir = '../'
sys.path.append(base_dir)
import os
from utils import *

import joblib
import os
import numpy as np
import pandas as pd
from gensim.models import Word2Vec
from gensim.similarities.annoy import AnnoyIndexer


from annoy import AnnoyIndex
import polars as pl
import implicit
from src.eval import model_eval

import scipy.sparse as sps
from utils import str2list
from src.config import raw_data_session_id_dir, candidate_file_name
from lightgbm import LGBMRanker
from lightgbm import early_stopping
from utils import *


/home/jupyter/.cache/pypoetry/virtualenvs/kdd-2023-KklMGVX0-py3.8/lib/python3.8/site-packages/implicit/gpu/__init__.py:13: UserWarning: CUDA extension is built, but disabling GPU support because of 'Cuda Error: no CUDA-capable device is detected (/project/./implicit/gpu/utils.h:71)'
  warnings.warn(


# Config 

In [2]:
candidate_file_name

'{task}_{data_type}_{model_version}_{model_for_eval}_top{topn}.parquet'

In [3]:
debug = False

eval_cg = False

load_train_eval_data = False
if debug:
    SAMPLE_NUM = 1000
else:
    SAMPLE_NUM = None

candidate_path = '../data/candidates/'
model_dir = '../model_training'
ranker_train_data_dir = '../data/rank_train_data'
# train_data_dir = '.'
# test_data_dir = '.'
task = 'task1'
w2v_model_version = 'w2v_v3'
nic_model_version = 'nic'
# rank_model_version = 'rank_lgbm_v2'

# rank_model_dir = os.path.join(model_dir, rank_model_version)
model_for_eval = True
w2v_topn=100
nic_topn=100
# PREDS_PER_SESSION = 100

# num_tree = 100
# # target locales: locales needed for task1
target_locals = ["DE", 'JP', 'UK']

# submit_file = f'submission_{task}_ALS.parquet'
num_tree = 100
w2v_model_dir = os.path.join(model_dir, w2v_model_version)
w2v_model_file = os.path.join(w2v_model_dir, f"{model_for_eval}.model")
annoy_index_file = os.path.join(w2v_model_dir, f"{str(num_tree)}_{model_for_eval}.index")

In [4]:
! mkdir {ranker_train_data_dir}

mkdir: cannot create directory ‘../data/rank_train_data’: File exists


In [5]:
# model_dir

# Original data 

In [6]:

# eval_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_eval.parquet'), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))

# # df_sess.head(3).collect()
# test_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, 'sessions_test_task1.parquet'), n_rows=SAMPLE_NUM).with_columns(pl.col('prev_items').apply(str2list))


# Get candiadtes

In [7]:
def get_all_candidates(data_type, task=task, 
                       w2v_model_version=w2v_model_version, 
                       nic_model_version=nic_model_version
                      ,model_for_eval=model_for_eval,
                      w2v_topn=w2v_topn
                      , nic_topn=nic_topn):
    
    w2v_file = os.path.join(candidate_path, 
                           candidate_file_name.format(
                    task=task
                , data_type=data_type
                , model_version=w2v_model_version
                , model_for_eval=model_for_eval
                , topn=w2v_topn
                           ))
    nic_file = os.path.join(candidate_path,
                candidate_file_name.format(
                    task=task
                    , data_type=data_type
                    , model_version=nic_model_version
                    , model_for_eval=model_for_eval
                    , topn=nic_topn
                           ))
    if data_type == 'test':
        original_file_name = f"sessions_{data_type}_{task}.parquet"
    else:
        original_file_name = f"sessions_{data_type}.parquet"
    original_pl = pl.scan_parquet(os.path.join(base_dir, raw_data_session_id_dir, original_file_name), n_rows=SAMPLE_NUM).filter(pl.col('locale').is_in(target_locals)).with_columns(pl.col('prev_items').apply(str2list))
    w2v_pl = pl.scan_parquet(w2v_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    nic_pl = pl.scan_parquet(nic_file, n_rows=SAMPLE_NUM)#.with_columns(pl.col('prev_items').apply(str2list))
    
    # get w2v weight
    w2v_pl = w2v_pl.with_columns(
        pl.lit([list(range(w2v_topn, 0, -1))]).alias('w2v_weight')
    )
    get_w2v_weight = pl.element().rank()*0
    nic_pl = nic_pl.with_columns(
        pl.col('next_item_prediction').arr.eval(get_w2v_weight, 
                                                # parallel=True
                                               ).alias('w2v_weight').cast(pl.List(pl.Int64))
    )
    cols = ['session_id', 'next_item_prediction', 'w2v_weight']
    combined_pl = w2v_pl.select(cols).join(nic_pl.select(cols), how='left', on='session_id', suffix='_nic')

    # combined_pl = combined_pl.with_columns(
    #     pl.concat_list([pl.col('next_item_prediction'), pl.col('next_item_prediction_nic')]).alias('next_item_prediction'), 
    #     pl.concat_list([pl.col('w2v_weight'), pl.col('w2v_weight_nic')]).alias('w2v_weight'), 
    # ).select(
    #     pl.all().exclude(['next_item_prediction_nic', 'w2v_weight_nic'])
    # )
    explode_cols = ['next_item_prediction', 'w2v_weight']
    combined_pl = (
            pl.concat([w2v_pl.select(cols).explode(explode_cols), nic_pl.select(cols).explode(explode_cols)], how='vertical')
                .groupby(['session_id', 'next_item_prediction'])
                .agg(
                    pl.col('w2v_weight').max().alias('w2v_weight')
                )
                .groupby('session_id')
                .agg(
                    pl.all()
                )
    )
    combined_pl = (
        combined_pl.join(original_pl, how='left', on='session_id')
            # .with_columns(
            #     pl.col('prev_items')
            # )
    )
    return combined_pl


In [8]:
train_cg_pl = get_all_candidates(data_type='train')
eval_cg_pl = get_all_candidates(data_type='eval')
test_cg_pl = get_all_candidates(data_type='test')

# Eval candidate generation 

In [9]:
if eval_cg:
    eval_topn = 300

    col = f"recall@{eval_topn}"

    eval_final = (
            eval_cg_pl
            .lazy()
            .with_columns(
                pl.col('next_item_prediction').cast(pl.List(pl.Utf8))
            )
            .with_columns(
                pl.concat_list([pl.col('next_item'), pl.col('next_item_prediction')]).alias('mrr')
            )
            ).with_columns(
                pl.col('next_item_prediction').arr.head(eval_topn).arr.contains(pl.col('next_item')).mean().alias(col)

            )
    final_res = eval_final.select(
            pl.count().alias('total_sessions')
            , pl.col(col).mean()

        ).collect()
    print(final_res)
    del final_res

# Feature Process 

## Load NIC, W2V Model 

In [10]:
nic_model = (
    pl.scan_parquet('../model_training/next_item_counter_v2/nic_model.parquet')
        .explode(['next_item_prediction', 'next_item_weight'])
        .select(
            pl.all().exclude('item')
            , pl.col('item').alias('last_prev_item')
        )
            )
print(nic_model.schema)



# w2v_model_file = '../model_training/v2/w2v.model'
w2vec = Word2Vec.load(w2v_model_file)
annoy_index = AnnoyIndexer()
annoy_index.load(annoy_index_file)

{'next_item_prediction': Utf8, 'next_item_weight': Float64, 'last_prev_item': Utf8}


## get features 

In [11]:
# target_df = train_candidates
# data_type = 'train'

def get_w2v_simi(x, col='last_prev_item'):
    try:
        if isinstance(x[col], str):
            target = [x[col]]
        else:
            target = x[col]
        simi = w2vec.wv.n_similarity([x['next_item_prediction']],
                                                  target
                                                 )
    except Exception as e:
        print(e)
        # print(x[col])
        simi = 0
    return simi

def get_feature(target_df, data_type, nic_model=nic_model, negative_sampling=None):

    target_df = target_df.with_columns(
                    pl.col('prev_items').arr.lengths().alias('prev_length')
                                      ).explode(['next_item_prediction', 'w2v_weight'])

    if data_type != 'test':
        target_df = (
            target_df
                .with_columns(
                    pl.when(pl.col('next_item')==pl.col('next_item_prediction')).then(1).otherwise(0).alias('target')
                )
        )

    if negative_sampling is not None:
        target_df = target_df.collect()
        pos = target_df.filter(pl.col('target')==1)
        neg = target_df.filter(pl.col('target')==0).sample(frac=negative_sampling)
        target_df = pl.concat([pos, neg], how='vertical')
        print(target_df.select('target').mean())

    target_df = (
            target_df.lazy().with_columns(
                    pl.col('prev_items').arr.get(-1).alias('last_prev_item')
                ).join(nic_model, how='left', on=['last_prev_item', 'next_item_prediction'])
                .with_columns(
                    pl.when(pl.col('next_item_weight').is_null()).then(0).otherwise(pl.col('next_item_weight')).alias('next_item_weight')
                    , pl.struct(["next_item_prediction", "last_prev_item"]).apply(
                        lambda x: get_w2v_simi(x, col='last_prev_item')).alias('last_item_similarity').cast(pl.Float32)
                    # , pl.struct(["next_item_prediction", "prev_items"]).apply(
                    #     lambda x: get_w2v_simi(x, col='prev_items')).alias('prev_item_similarity').cast(pl.Float32)
                    , pl.when(pl.col('locale')=='DE').then(1).when(pl.col('locale')=='DE')
                        .then(2)
                        .otherwise(3).alias('locale')
                ).sort('session_id')

    )
    return target_df
# target_df.head(3).collect()

In [12]:
# %%time 
# sample_pl = get_feature(target_df=train_cg_pl, data_type='train', #negative_sampling=0.1
#                        ).collect()

In [13]:
# %%time 
# sample_pl = get_feature(target_df=train_cg_pl, data_type='train',# negative_sampling=0.1
#                        ).collect()

In [14]:
9

9

In [ ]:
sampled_train_candidates = get_feature(target_df=train_cg_pl, data_type='train', negative_sampling=0.1)
eval_candidates = get_feature(target_df=eval_cg_pl, data_type='eval')
test_candidates = get_feature(target_df=test_cg_pl, data_type='test')


# Save Data 

In [ ]:
sampled_train_candidates = sampled_train_candidates.collect()
eval_candidates = eval_candidates.collect()
test_candidates = test_candidates.collect()

In [ ]:
print('Sampled train data')
print(sampled_train_candidates.shape)
print(sampled_train_candidates['target'].mean())
print('Eval data')
print(eval_candidates.shape)
print(eval_candidates['target'].mean())
print('Test data')
print(test_candidates.shape)

In [ ]:
ranker_train_data_dir

In [ ]:
if not debug:
    sampled_train_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'train.parquet'))
    eval_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'eval.parquet'))
    test_candidates.write_parquet(os.path.join(ranker_train_data_dir, 'test.parquet'))